# What is this notebook about?

In ionospheric studies it is often usefull work with geomagnetic coordinates since they track geomagnetic field and some structures follow the geomagnetic field.

![geo_mag_grid](https://agupubs.onlinelibrary.wiley.com/cms/asset/9c967c57-04dc-4087-9c22-575982058867/jgra16515-fig-0003.png)

This notebook will guide you through the conversion of the geographic coordinates to geomagnetic ones. 

# Requirements

The notebook relies on data from `lat_lon_geo_mag_v2010.csv` - make sure it is in the same directory.

In [9]:
grid_file = "lat_lon_geo_mag_v2010.csv" 

In [16]:
import numpy as np
from pathlib import Path
from numpy.typing import NDArray

def read_geomagnetic_grid(file_path: Path) -> NDArray:
    """
    Reads the geomagnetic coordinate grid from a file.

    Assumes structure as:

    #Lat, Lon, MLat, MLon
     -90.0    -180.0   -74.5    18.0
     -90.0    -179.0   -74.5    18.0
     -90.0    -178.0   -74.5    18.0
     -90.0    -177.0   -74.5    18.0
    
    Returns:
        data: numpy array with columns [Lat, Lon, MLat, MLon]
    """
    return np.loadtxt(file_path, comments="#")


def bilinear_interpolate(
    lat: float, 
    lon: float, 
    grid: NDArray, 
    latstep: float=1.0, 
    lonstep: float=1.0
)-> tuple[float, float]: 
    """
    Interpolates the geomagnetic coordinates for a given geographical location.

    Args:
        lat (float): Latitude in degrees
        lon (float): Longitude in degrees
        grid (np.ndarray): Numpy array with columns [Lat, Lon, MLat, MLon]
        latstep (float): Latitude in degrees
        lonstep (float): Longitude in degrees

    Returns:
        tuple: (interpolated MLat, MLon)
    """

    # Handle poles: return exact value without interpolation
    if lat == 90.0 or lat == -90.0:
        # Find the closest match (lon may be any value at poles)
        pole_rows = grid[grid[:, 0] == lat]
        if pole_rows.size == 0:
            raise ValueError(f"No data found for pole at latitude {lat}")
        # Return the first match (all longitudes at the pole are equivalent)
        return pole_rows[0][2], pole_rows[0][3]

    # -180 and 180 is the same point 
    lon = ((lon + 180) % 360) - 180

    # Round lat/lon down to nearest integer grid
    lat0 = np.floor(lat)
    lon0 = np.floor(lon)
    lat1 = lat0 + 1
    lon1 = lon0 + 1

    # Get four corner points
    mask = (
        ((grid[:, 0] == lat0) & (grid[:, 1] == lon0)) |
        ((grid[:, 0] == lat0) & (grid[:, 1] == lon1)) |
        ((grid[:, 0] == lat1) & (grid[:, 1] == lon0)) |
        ((grid[:, 0] == lat1) & (grid[:, 1] == lon1))
    )
    neighbors = grid[mask]

    if neighbors.shape[0] != 4:
        raise ValueError("Could not find a full 2x2 grid for interpolation.")

    # Sort neighbors by (lat, lon)
    neighbors = sorted(neighbors, key=lambda x: (x[0], x[1]))
    Q11 = neighbors[0]
    Q12 = neighbors[1]
    Q21 = neighbors[2]
    Q22 = neighbors[3]

    # Bilinear interpolation formula
    def interpolate(x, y, Q11, Q12, Q21, Q22):
        x1, y1 = Q11[0], Q11[1]
        x2, y2 = Q22[0], Q22[1]
        if x2 == x1 or y2 == y1:
            return Q11[2], Q11[3]  # Avoid division by zero
        fx = (x - x1) / (x2 - x1)
        fy = (y - y1) / (y2 - y1)
        
        _y = (
            Q11[2] * (1 - fx) * (1 - fy) +
            Q21[2] * fx * (1 - fy) +
            Q12[2] * (1 - fx) * fy +
            Q22[2] * fx * fy
        )
        _x = (
            Q11[3] * (1 - fx) * (1 - fy) +
            Q21[3] * fx * (1 - fy) +
            Q12[3] * (1 - fx) * fy +
            Q22[3] * fx * fy
        )
        return _y, _x

    return interpolate(lat, lon, Q11, Q12, Q21, Q22)


In [17]:
file_path = grid_file  # Replace with your file path
grid = read_geomagnetic_grid(file_path)
# se line "-89.0     -60.0   -73.6    17.6" in a file
lat, lon = -89.0, -60.0
mlat, mlon = bilinear_interpolate(lat, lon, grid)
print(f"Interpolated geomagnetic coordinates for ({lat}, {lon}) → MLat: {mlat}, MLon: {mlon}")
print(f"Geomagnetic coordinates for (-89.0, -60.0) → MLat: -73.6, MLon: 17.6")

Interpolated geomagnetic coordinates for (-89.0, -60.0) → MLat: -73.6, MLon: 17.6
Geomagnetic coordinates for (-89.0, -60.0) → MLat: -73.6, MLon: 17.6


# Perform test

In [18]:
import numpy as np

def test_bilinear_interpolation_against_file(file_path, tolerance=1e-6):
    """
    Tests the bilinear interpolation function against ground truth in the file.

    Args:
        file_path (str): Path to the file containing [Lat, Lon, MLat, MLon]
        tolerance (float): Acceptable difference between interpolated and true values

    Raises:
        AssertionError: If any interpolated value deviates beyond the tolerance
    """
    data = read_geomagnetic_grid(file_path)

    passed = 0
    failed = 0

    for i in range(data.shape[0]):
        lat, lon, true_mlat, true_mlon = data[i]

        try:
            interp_mlat, interp_mlon = bilinear_interpolate(lat, lon, data)
            if (
                abs(interp_mlat - true_mlat) > tolerance or
                abs(interp_mlon - true_mlon) > tolerance
            ):
                print(f"FAILED at index {i}:")
                print(f"  Input:  lat={lat}, lon={lon}")
                print(f"  Truth:  MLat={true_mlat}, MLon={true_mlon}")
                print(f"  Interp: MLat={interp_mlat}, MLon={interp_mlon}")
                failed += 1
            else:
                passed += 1
        except ValueError as e:
            print(f"Interpolation skipped at index {i} and lat={lat}, lon={lon} due to missing neighbors: {e}")
            failed += 1

    print(f"\nTest Summary: {passed} passed, {failed} failed")

In [19]:
test_bilinear_interpolation_against_file(grid_file)


Test Summary: 65341 passed, 0 failed
